# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [62]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [63]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [64]:
import queue

def astar(problem: Problem):
    pQueue = queue.PriorityQueue()
    state = problem.initial_state
    g = 0 #cost_so_far
    h = problem.heuristic(state)
    pQueue.put((g + h, g, state, []))
    visited = set()
    no_states = 1
    list_of_actions = []
    actions = problem.available_actions(state)
    while not pQueue.empty():
        ex, cost_so_far, item, list_of_actions = pQueue.get()
        no_states += 1
        
        if problem.is_goal(item):
            print(no_states)
            return list_of_actions
        
        for i in actions:
            new_item = problem.do_action(item, i)
            if new_item not in visited:
                cost_so_far += problem.action_cost(item, i)
                h = problem.heuristic(new_item)
                pQueue.put((cost_so_far + h, cost_so_far, new_item, list_of_actions+[i]))
                visited.add(new_item)
    return None

Now lets test your code in the vacuum world!

In [65]:
astar(VacuumProblem())

6


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [66]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

8


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [67]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

5


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

The best is the VacuumProblem2, which visits only 5 states. At the first moment, I didn't expect such a result, because this way of finding the solution isn't admissible. Nevertheless, when I have thought about it for a longer time I realize that it has sense. The VacuumProblem2 heuristic makes a huge penalty for not sucking the dirt if the room is dirty. Therefore, in the priority queue all the states which are not accessing by sucking, are at the end of the queue, which forced the algorithm to do action suck whenever the vacuum cleaner is in the dirty room. The strategy to suck dirt every time it is possible is the best for this problem.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [68]:
import math

class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return (7,2,4,5,0,6,8,3,1)
        
    def available_actions(self, state):
        return ["Up", "Right", "Down", "Left"]
        
    def do_action(self, state, action):
        lstate = list(state)
        pos = 0
        ret = None
        for i in range(0,9):
            if state[i] == 0:
                pos = i
                break
             
        if action == "Up":
            if pos > 2:
                lstate[pos], lstate[pos - 3] = lstate[pos - 3], lstate[pos]
            ret = tuple(lstate)
            
        elif action == "Right":
            if pos != 8 and pos != 5 and pos != 2:
                lstate[pos], lstate[pos + 1] = lstate[pos + 1], lstate[pos]
            ret = tuple(lstate)
            
        elif action == "Down":
            if pos < 6:
                lstate[pos], lstate[pos + 3] = lstate[pos + 3], lstate[pos]
            ret = tuple(lstate)
            
        elif action == "Left":
            if pos != 0 and pos != 3 and pos != 6:
                lstate[pos], lstate[pos - 1] = lstate[pos - 1], lstate[pos]
            ret = tuple(lstate)
            
        return ret

    def is_goal(self, state) -> bool:
        if state == (0,1,2,3,4,5,6,7,8):
            return True
        return False
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        suma = 0
        for i in range(1,9):
            if state[i] != i:
                suma += 1
        return suma

**Prove that this heuristic is admissible.**

1)By moving the blank tile (tile with number 0), we change the position and position of one of the tiles.

2)Every tile numbered from 1 to 8 which isn't at its goal position must be at least once moved.

3)This heuristic approach count how many tiles (excluding blank tile) aren't at their positions and returns the sum - let's name it s.

4)The number of swaps we have to do is >= s, therefore this approach is admissible.

In [69]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:
        suma = 0
        for i in range(1,9):
            pos = 0
            while(i != state[pos]):
                pos+=1
            suma += abs(pos%3 - i%3) + abs(math.floor(i/3) - math.floor(pos/3))
        return suma


**Prove that this heuristic is admissible.**

1)The function counts the sum of Manhattan distances of all the tiles (excluding blank tile) to their goal positions.

2)Every move (going from one state to another) can change places only of one of the tiles with the blank tile.

3)We need at least d moves to set the tile to its goal position if its distance equals d.

4)Therefore this heuristic is admissible.

In [70]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        suma = 0
        for i in range(1,9):
            pos = 0
            while(i != state[pos]):
                pos+=1
            suma += max(abs(pos%3 - i%3), abs(math.floor(i/3) - math.floor(pos/3)))
        return suma

**Prove that this heuristic is admissible.**

1)The function for class PuzzleProblem2 returns the sum of distances of all tiles (excluding the blank one) from their goal positions according to Chebyshev distance measure.

2)It always returns estimated cost which is less or equal those from PuzzleProblem1 ( max(a,b) <= a + b).

3)Therefore if PuzzleProblem1 is admissible, then PuzzleProblem2 also is.

Run your heuristics on the given problem.

In [71]:
plan0 = astar(PuzzleProblem())
plan1 = astar(PuzzleProblem1())
plan2 = astar(PuzzleProblem2())

print("Plan to be executed", len(plan0))
print("Plan to be executed", len(plan1))
print("Plan to be executed", len(plan2))

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

92332
17603
47564
Plan to be executed 26
Plan to be executed 26
Plan to be executed 26
Is plan0==plan1? True
Is plan0==plan2? True
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

The best is the PuzzleProblem1 heuristic. For this heuristic, the A-star algorithm visits the least number of states. It is because for this heuristic estimated costs are higher than in the other 2 heuristics (*), but still not overestimated. The high estimated cost of achieving the goal state helps us to eliminate those states which are further from meta, than others.

* The cost of PuzzleProblem1 is >= cost od PuzzleProblem, because in PP1 we not only count the misplaced tiles but if their Manhattan distance from the goal position is> 1, then we add bigger numbers to the sum
* The cost of PuzzleProblem1 is >= cost od PuzzleProblem2, because in PP1 the cost equals a+b (a - the vertical distance, b - the level distance) and in PP2 it equals max(a,b). a+b >= max(a,b)

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.